### Short Word - Character Level RNN

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str + label_str)))
vocab_size = len(char_vocab)
print ('문자 집합의 크기 : {}'.format(vocab_size))

문자 집합의 크기 : 5


In [3]:
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [4]:
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [5]:
index_to_char = {}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [6]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [7]:
x_data = [x_data]
y_data = [y_data]

x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [8]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([1, 5, 5])
레이블의 크기 : torch.Size([1, 5])


C:\Users\jamco\AppData\Local\Temp\ipykernel_11248\306303581.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  X = torch.FloatTensor(x_one_hot)


In [9]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, output_size, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [10]:
net = Net(input_size, hidden_size, output_size)

In [11]:
outputs = net(X)
print(outputs.shape)

torch.Size([1, 5, 5])


In [12]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [13]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, input_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis=2)
    result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.6452080011367798 prediction:  [[3 3 3 3 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  lllll
1 loss:  1.4056836366653442 prediction:  [[4 4 4 0 2]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppp!e
2 loss:  1.2222589254379272 prediction:  [[4 4 4 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppp!!
3 loss:  1.0258066654205322 prediction:  [[4 4 4 0 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppp!!
4 loss:  0.8298776745796204 prediction:  [[4 4 3 2 2]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplee
5 loss:  0.6569910049438477 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
6 loss:  0.5194292068481445 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
7 loss:  0.40987300872802734 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.318220853805542 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.23626144230365753 prediction:  [[4 4 3 2 0]]

### Long Word - Character Level RNN

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim

In [15]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [16]:
char_set = list(set(sentence))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

{'f': 0, 'o': 1, 'h': 2, 'i': 3, 'd': 4, 'w': 5, 's': 6, ',': 7, '.': 8, 'b': 9, 'u': 10, 't': 11, 'm': 12, 'a': 13, 'g': 14, 'r': 15, 'l': 16, 'k': 17, 'y': 18, 'p': 19, 'c': 20, 'e': 21, "'": 22, 'n': 23, ' ': 24}
문자 집합의 크기 : 25


In [17]:
hidden_size = dic_size
sequence_length = 10  #random
learning_rate = 0.1

In [18]:
x_data, y_data = [], []

for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [19]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [20]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [21]:
net = Net(dic_size, hidden_size, 2)

In [22]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [23]:
outputs = net(X)
print(outputs.shape)

torch.Size([170, 10, 25])


In [24]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]
    print(predict_str)

ob'i'''''''i''b'''''''''''i'''''''i'''pb'''''''''b''''''''b''b'''''''i'''''''''p''''i'''''''''''b'''''''''''''''''''i''''''b''''''''''b''b''r'b'''''''''''''''b''''''''i'''b'''b'''
                                                                                                                                                                                   
tosootsoosorootsooaotosooototoootooootsaoto.oslots.ooooos.toaoooooososoooososlsoootooosrrto.ooootstsrot.gos.oosoo.oo.otsaost.aoros.oo.roao.oottoootooo.rst.rs.r.oo.ros.ooootooosros
eototttotettottttttttttetototoettotottttttttottottotttotottottoettttttottttteetotttotttttottottttotttetoettotttotttetotottttotttetottttottoetttoettototottttettotttettettoetotttott
 ldldlt tdtdldt  td  d td t t  hd     t tdtd t   t   d t     td tde  t     t td dht t   t t   edl  tdlt  l    t t     t t d   td t    l  td tdth tdld   dl  t t l tdtd  dolto hde  
   hdhth h ih  hh h to hdh h   h h   hdh h hrghd thhhlhhh td ih kh h ih h th  ho,h h h  th t    ih  